In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("../input/shopee-product-matching/train.csv")
test = pd.read_csv("../input/shopee-product-matching/test.csv")
sub = pd.read_csv("../input/shopee-product-matching/sample_submission.csv")
train.head()

In [ ]:
#找出每欄位unique有幾類
#for col in train.columns:
    #print(col + ":" + str(len(train[col].unique())))

In [ ]:
#正確答案
#tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
#train['target'] = train.label_group.map(tmp)
#print('train shape is', train.shape )
#train.head()

In [ ]:
#以tiltes分類可分成33117類，預測一樣的title會有一樣的label_group

#labels = train.groupby("title")["image"].count().reset_index()
#labels.columns=["title","image_num"]
#titles = labels.sort_values("image_num")
#titles

# 計算TFIDF

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english',binary = True, max_features = 25000)
tfidf_matrix = vectorizer.fit_transform(test.title)
print(type(tfidf_matrix))
print(tfidf_matrix.shape)

tfidfs = tfidf_matrix.toarray()
words = vectorizer.get_feature_names()
#pd.DataFrame(tfidfs,columns=words)
#tfidfs

# cosine_similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#計算相似度
cosine_similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
#print(cosine_similarity_matrix)

In [ ]:
print(cosine_similarity_matrix.shape)

In [ ]:
'''# cosine_sim[0] 為第一個文件與其他文件的相似度
cosine_sim = cosine_similarity_matrix[0]

# 去除第一個文件(查詢文件)
cosine_sim = cosine_sim[1:]

# 將文件編號與文件相似度打包為tuple
sim_scores = list(enumerate(cosine_sim))

# 以相似度降冪排列
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# 設定threshold取出相似的title
top10 = sim_scores[:10]

threshold = 0.8
print("第一句", train.title[0])

for i in sim_scores:
    idx, score = i[0] , i[1]
    
    if score > threshold:
        #train["title_pred"]= train.posting_id[idx]
        print("與其相似為:",idx, train.title[idx],score)
    else:
        break
'''

In [ ]:
def similarity(title):
    cntx = test[['title', 'posting_id']]
    #Reverse mapping of the index
    indices = pd.Series(test.index, index = test['title']).drop_duplicates()
         
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarity_matrix[idx-1]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    uplimit = len(test['title']) if len(test['title']) < 50 else 50
    sim_scores = sim_scores[0:uplimit]
    post_indices = [i[0] for i in sim_scores if i[1] > .8]
    recommend = cntx.iloc[post_indices]
    output=[]
    output.append(cntx.iloc[idx,1])
    for index, row in recommend.iterrows():
        output.append(row['posting_id'])
    return output

    #return ' '.join( np.unique(output))

In [ ]:
'''
# The Top 3 similar matches
#threshold = 0.8

def similarity(title):
    cntx = train[['title', 'posting_id']]
    #Reverse mapping of the index
    indices = pd.Series(train.index, index = train['title']).drop_duplicates()
         
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarity_matrix[idx-1]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    #sim_scores = sim_scores[0:1]
    
    for i in sim_scores:
        book_indices,scores  = i[0],i[1]
        if scores > threshold:
            recommend = cntx.iloc[book_indices]
            output=''
            for index, row in recommend.iteritems():
                output+=row['posting_id']+" "
        else:
            break 
        return output
'''
        

### Use image phash

In [ ]:
#正確答案
#tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
#train['target'] = train.label_group.map(tmp)
#print('train shape is', train.shape )
#train.head()

In [ ]:
#train['f1'] = train.apply(getMetric('phash_pred'),axis=1)
#print('CV score for baseline =',train.f1.mean())

In [ ]:
'''def getMetric(col):
    def f1score(row):
        n = len(np.intersect1d(row.target, row[col]))
        return 2*n / (len(row.target) + len(row[col]))
    return f1score'''

In [ ]:
#一樣image phash match在一起

tmp2 = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['phash_pred'] = test.image_phash.map(tmp2)

In [ ]:
#def tostr(x):
#    return ' '.join(x)
#test['phash_pred'] = test.phash_pred.map(tostr)


In [ ]:
test['title_pred']=test['title'].apply(similarity)
test.head()

In [ ]:
# Function to combine predictions
def combine_predictions(row):
    x = np.concatenate([row['phash_pred'], row['title_pred']])
    return ' '.join( np.unique(x) )

In [ ]:
test['matches'] = test.apply(combine_predictions, axis = 1)
test.head()

In [ ]:
test[['posting_id', 'matches']].head()

In [ ]:
test[['posting_id', 'matches']].to_csv('submission.csv', index = False)
sub = pd.read_csv('submission.csv')
sub.head()

In [ ]:
#def tostr(x):
#    return ' '.join(x)
#train['phash_pred'] = train.phash_pred.map(tostr)
#train['target'] = train.target.map(tostr)
#train.head()